## Generate a simple social graph

[python-igraph Manual](http://igraph.org/python/doc/tutorial/tutorial.html)

In [15]:
from igraph import *

In [12]:
print(igraph.__version__)

0.7.1


In [24]:
g = Graph()

In [25]:
print(g)

IGRAPH U--- 0 0 --


In [26]:
g.add_vertices(3)

In [27]:
g.add_edges([(0,1), (1,2)])

In [23]:
print(g)

IGRAPH U--- 0 0 --


In [29]:
g.add_edges([(2,0)])
g.add_vertices(3)
g.add_edges([(2,3),(3,4),(4,5),(5,3)])
print(g)

IGRAPH U--- 6 7 --
+ edges:
0--1 1--2 0--2 2--3 3--4 4--5 3--5


In [30]:
g.add_edges([(2,0)])
print(g) >>> multi-edges

IGRAPH U--- 6 8 --
+ edges:
0--1 1--2 0--2 2--3 3--4 4--5 3--5 0--2


In [31]:
g.get_eid(2,3)

3

In [32]:
g.delete_edges(3)

In [33]:
summary(g)

IGRAPH U--- 6 7 -- 


Graph.Tree() generates a regular tree graph. The one that we generated has 127 vertices and each vertex (apart from the leaves) has two children (and of course one parent).(简单的说就是每个点恰有两个分支) No matter how many times you call Graph.Tree(), the generated graph will always be the same if you use the same parameters:

In [34]:
g = Graph.Tree(127, 2)

In [35]:
summary(g)

IGRAPH U--- 127 126 -- 


In [39]:
g2 = Graph.Tree(127, 2)
g2.get_edgelist() == g.get_edgelist()

True

In [42]:
g2.get_edgelist()[0:10]  # 画出来就知道了

[(0, 1),
 (0, 2),
 (1, 3),
 (1, 4),
 (2, 5),
 (2, 6),
 (3, 7),
 (3, 8),
 (4, 9),
 (4, 10)]

Graph.GRG() generates a geometric random graph: n points are chosen randomly and uniformly inside the unit square (单位正方形) and pairs of points closer to each other than a predefined distance d are connected by an edge. In our case, n is 100 and d is 0.2. Due to the random nature of the algorithm, chances are that the exact graph you got is different from the one that was generated when I wrote this tutorial, hence the values above in the summary will not match the ones you got. This is normal and expected. Even if you generate two geometric random graphs on the same machine, they will be different for the same parameter set:

In [41]:
g = Graph.GRG(100, 0.2)
summary(g)

IGRAPH U--- 100 551 -- 
+ attr: x (v), y (v)


In [43]:
g2 = Graph.GRG(100, 0.2)
g.get_edgelist() == g2.get_edgelist()

False

isomorphic() tells you whether two graphs are isomorphic(同构) or not. In general, it might take quite a lot of time, especially for large graphs, but in our case, the answer can quickly be given by checking the degree distributions of the two graphs.

In [45]:
g.isomorphic(g2)

False

## [Setting and retrieving attributes](http://igraph.org/python/doc/tutorial/tutorial.html#setting-and-retrieving-attributes)

igraph uses vertex and edge IDs in its core. These IDs are integers, starting from zero, and they are always continuous at any given time instance during the lifetime of the graph. This means that whenever vertices and edges are deleted, a large set of edge and possibly vertex IDs will be renumbered to ensure the continuity. Now, let us assume that our graph is a social network where vertices represent people and edges represent social connections between them. One way to maintain the association between vertex IDs and say, the corresponding names is to have an additional Python list that maps from vertex IDs to names. The drawback of this approach is that this additional list must be maintained in parallel to the modifications of the original graph. Luckily, igraph knows the concept of attributes, i.e., auxiliary objects associated to a given vertex or edge of a graph, or even to the graph as a whole. Every igraph Graph, vertex and edge behaves as a standard Python dictionary in some sense: you can add key-value pairs to any of them, with the key representing the name of your attribute (the only restriction is that it must be a string) and the value representing the attribute itself. (即attribute用key表示)

In [108]:
g = Graph([(0,1), (0,2), (2,3), (3,4), (4,2), (2,5), (5,0), (6,3), (5,6)])
g.summary()

'IGRAPH U--- 7 9 -- '

Now, let us assume that we want to store the names, ages and genders of people in this network as vertex attributes, and for every connection, we want to store whether this is an informal friendship tie or a formal tie. Every Graph object contains two special members called vs and es, standing for the sequence of all vertices and all edges, respectively. If you try to use vs or es as a Python dictionary, you will manipulate the attribute storage area of the graph:·

In [109]:
g.vs

In [110]:
g.vs["name"] = ["Alice", "Bob", "Claire", "Dennis", "Esther", "Frank", "George"] # .vs 是给vertical加attribute

In [111]:
g.vs["age"] = [25, 31, 18, 47, 22, 23, 50]

In [112]:
g.vs["gender"] = ["f", "m", "f", "m", "f", "m", "m"]

In [113]:
g.es["is_formal"] = [False, False, True, True, True, False, True, False, False] # .es 是给edge加attribute

Whenever you use vs or es(EdgeSeq) as a dictionary, you are assigning attributes to all vertices/edges of the graph. However, you can simply alter the attributes of vertices and edges individually by indexing vs or es with integers as if they were lists (remember, they are sequences, they contain all the vertices or all the edges). When you index them, you obtain a Vertex or Edge object, which refers to (I am sure you already guessed that) a single vertex or a single edge of the graph. Vertex and Edge objects can also be used as dictionaries to alter the attributes of that single vertex or edge:

In [61]:
g.es[0]

igraph.Edge(<igraph.Graph object at 0x10c38a228>, 0, {'is_formal': False})

In [62]:
g.vs[0]

igraph.Vertex(<igraph.Graph object at 0x10c38a228>, 0, {'name': 'Alice', 'age': 25, 'gender': 'f'})

In [63]:
g.es[0].attributes()

{'is_formal': False}

In [66]:
g.es[0]["is_formal"] = True # 重新赋值attribute

In [67]:
g.es[0]

igraph.Edge(<igraph.Graph object at 0x10c38a228>, 0, {'is_formal': True})

In [70]:
g.es

Not too surprisingly, Graph objects themselves can also behave as dictionaries:

In [71]:
g["date"] = "2009-01-10"

In [72]:
print(g["date"])

2009-01-10


Finally, it should be mentioned that attributes can be deleted by the Python keyword `del` just as you would do with any member of an ordinary dictionary:

In [74]:
g.vs[3]["foo"] = "bar"

In [75]:
g.vs["foo"]

[None, None, None, 'bar', None, None, None]

In [76]:
del g.vs["foo"]

In [77]:
g.vs["foo"]

KeyError: 'Attribute does not exist'

## [Structural properties of graphs](http://igraph.org/python/doc/tutorial/tutorial.html#structural-properties-of-graphs)

Besides the simple graph and attribute manipulation routines described above, igraph provides a large set of methods to calculate various structural properties of graphs. It is beyond the scope of this tutorial to document all of them, hence this section will only introduce a few of them for illustrative purposes. We will work on the small social network we built in the previous section.

Probably the simplest property one can think of is the vertex degree. The degree of a vertex equals the number of edges adjacent to it. In case of directed networks, we can also define in-degree (the number of edges pointing towards the vertex) and out-degree (the number of edges originating from the vertex). igraph is able to calculate all of them using a simple syntax:

In [78]:
g.degree()

[3, 1, 4, 3, 2, 3, 2]

If the graph was directed, we would have been able to calculate the in- and out-degrees separately using g.degree(type="in") and g.degree(type="out"). You can also pass a single vertex ID or a list of vertex IDs to degree() if you want to calculate the degrees for only a subset of vertices:

In [80]:
g.degree(6) # 最后一个点的degree

2

In [81]:
g.degree([2,3,4])

[4, 3, 2]

Besides degree, igraph includes built-in routines to calculate many other centrality properties, including vertex and edge betweenness (`Graph.betweenness()`, `Graph.edge_betweenness()`) or Google’s PageRank (`Graph.pagerank()`) just to name a few. Here we just illustrate edge betweenness:

In [82]:
g.edge_betweenness()

[6.0, 6.0, 4.0, 2.0, 4.0, 3.0, 4.0, 3.0, 4.0]

In [83]:
g.betweenness()

[5.0, 0.0, 5.5, 1.5, 0.0, 2.5, 0.5]

In [84]:
g.pagerank()

[0.1715187083669299,
 0.07002553879920158,
 0.20933537164407268,
 0.16151684644322287,
 0.11167544439518333,
 0.16265174994590004,
 0.11327634040548959]

Now we can also figure out which connections have the highest betweenness centrality with some Python magic:

In [86]:
ebs = g.edge_betweenness()
max_eb = max(ebs)
[g.es[idx].tuple for idx, eb in enumerate(ebs) if eb == max_eb] ## 称为Python magic 真不为过

[(0, 1), (0, 2)]

In [95]:
[g.es[idx] for idx, eb in enumerate(ebs) if eb == max_eb] ## 所以.tuple可以把对应的edge用两点连线方式表示出来

[igraph.Edge(<igraph.Graph object at 0x10c38a228>, 0, {'is_formal': True}),
 igraph.Edge(<igraph.Graph object at 0x10c38a228>, 1, {'is_formal': False})]

Most structural properties can also be retrieved for a subset of vertices or edges or for a single vertex or edge by calling the appropriate method on the VertexSeq, EdgeSeq, Vertex or Edge object of interest:

In [96]:
g.vs.degree()

[3, 1, 4, 3, 2, 3, 2]

In [97]:
g.es.edge_betweenness()

[6.0, 6.0, 4.0, 2.0, 4.0, 3.0, 4.0, 3.0, 4.0]

In [98]:
g.vs[2].degree()

4

## [Querying vertices and edges based on attributes](http://igraph.org/python/doc/tutorial/tutorial.html#querying-vertices-and-edges-based-on-attributes)

Imagine that in a given social network, you would like to find out who has the largest degree or betweenness centrality. You can do that with the tools presented so far and some basic Python knowledge, but since it is a common task to select vertices and edges based on attributes or structural properties, igraph gives you an easier way to do that:

In [114]:
g.vs.attribute_names() ### >>> ['name', 'age', 'gender']
g.vs.select(_degree = g.maxdegree())["name"]

['Claire']

The syntax may seem a little bit awkward for the first sight, so let’s try to interpret it step by step. select() is a method of VertexSeq and its sole purpose is to filter a VertexSeq based on the properties of individual vertices. The way it filters the vertices depends on its positional and keyword arguments. Positional arguments (the ones without an explicit name like `_degree` above) are always processed before keyword arguments as follows:

1. If the first positional argument is None, an empty sequence (containing no vertices) is returned:

In [115]:
seq = g.vs.select(None)
len(seq)

0

2. If the first positional argument is a callable object (i.e., a function, a bound method or anything that behaves like a function), the object will be called for every vertex that’s currently in the sequence. If the function returns True, the vertex will be included, otherwise it will be excluded:

In [116]:
graph = Graph.Full(10)
only_odd_vertices = graph.vs.select(lambda vertex: vertex.index % 2 == 1)
len(only_odd_vertices)

5

In [119]:
graph.vs.indices

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]